# API testing notebook

In [15]:
import requests
import base64
from io import BytesIO
from PIL import Image
import uuid

def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

URL_API = "https://toa4nqj3umza2o-5000.proxy.runpod.net/"

In [2]:
# basic route
print(requests.get(URL_API).text)

<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>


<title>toa4nqj3umza2o-5000.proxy.runpod.net | 502: Bad gateway</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" />


</head>
<body>
<div id="cf-wrapper">
    <div id="cf-error-details" class="p-0">
        <header class="mx-auto pt-10 lg:pt-6 lg:px-8 w-240 lg:w-full mb-8">
            <h1 class="inline-block sm:block sm:mb-2 font-light text-60 lg:text-4xl text-black-da

In [19]:
# info route

url = URL_API + "info"
image_path = "image/coconut.jpg"
encoded_image = encode_image_to_base64(image_path)
payload = {"image": encoded_image}

response = requests.post(url, json=payload)

print(response.text)

<!doctype html>
<html lang=en>
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>



In [ ]:
# next position

url = URL_API + "transform"

payload = {
    "input_images": {
        "input_img.jpg": encoded_image
    },
    "workflow": "img2img",
    "client_id": str(uuid.uuid4()),
    "params": {
        "input_image": "input_img.jpg" 
    },
    "coords": [0.0, 0.0]
}

response = requests.post(url, json=payload)

card_info = response.json()

print(card_info["info_text"])

base64_str = card_info["image_b64"].split(',')[1]
Image.open(BytesIO(base64.b64decode(base64_str)))

In [1]:
import requests
import base64
from io import BytesIO
from comfycaller import generate, get_media
from PIL import Image
import uuid

def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

URL_API = "https://toa4nqj3umza2o-5000.proxy.runpod.net/"

def load_b64(image_b64):
    image = Image.open(BytesIO(base64.b64decode(image_b64.split(',', 1)[-1]))).convert("RGB")
    return image

params = {
    "input_image": "input_img.jpg" 
}

image_path = "image/coconut.jpg"
encoded_image = encode_image_to_base64(image_path)
image = base64.b64decode(encoded_image)

input_images = {
    "input_img.jpg": image
}

outputs = generate("text2morph", None, input_images, str(uuid.uuid4()))

In [7]:
for k in outputs:
    output_types = list(outputs[k].keys())
    if 'gifs' in output_types:
        media_info = outputs[k]['gifs'][0]
        mp4_b64 = get_media(media_info['filename'], media_info['subfolder'], media_info['type'])

In [17]:
import base64
import os

base, extension = os.path.splitext(media_info['filename'])
file_path = f"my_superb_name{extension}"

with open(file_path, "wb") as mp4_file:
    mp4_file.write(mp4_b64)